In [1]:
import numpy as np
import pandas as pd
import glob
import scipy.misc
import tensorflow as tf
import scipy
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
BORED_APES_ADDRESS = "0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d"
KEY = "ckey_773ccb8db17c4884a9ff935d884"

# BORED APES

In [3]:
import requests
URL = f"https://api.covalenthq.com/v1/1/tokens/{BORED_APES_ADDRESS}/nft_metadata/123/?quote-currency=USD&format=JSON&key="
res = requests.get(URL + KEY)
res.json()

{'data': {'updated_at': '2022-03-07T16:36:27.978484782Z',
  'items': [{'contract_decimals': 0,
    'contract_name': 'BoredApeYachtClub',
    'contract_ticker_symbol': 'BAYC',
    'contract_address': '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d',
    'supports_erc': ['erc20'],
    'logo_url': 'https://logos.covalenthq.com/tokens/1/0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d.png',
    'type': 'nft',
    'nft_data': [{'token_id': '123',
      'token_balance': '1',
      'token_url': 'https://ipfs.io/ipfs/QmeSjSinHpPnmXmspMjwiXyN6zS4E9zccariGR3jxcaWtq/123',
      'supports_erc': ['erc20', 'erc721'],
      'token_price_wei': None,
      'token_quote_rate_eth': None,
      'original_owner': '0x643ce52422b668f0e226dd21b4bb8d3a1fabca51',
      'external_data': {'name': None,
       'description': None,
       'image': 'https://ipfs.io/ipfs/QmVP1tqb9jf6XCkZZXkqGfTAtS8KwXHKHvkePh62zyL65n',
       'image_256': 'https://image-proxy.svc.prod.covalenthq.com/256,fit,png/https://ipfs.io/ipfs/QmVP1tqb9jf

## Try an example

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 

image_url = res.json()["data"]["items"][0]["nft_data"][0]["external_data"]["image"]
Image(url=image_url)

In [5]:
img_data = requests.get(image_url).content
with open('images/123.jpg', 'wb') as handler:
    handler.write(img_data)

In [6]:
df = pd.DataFrame(columns=["image_id", "trait_type", "value"])

attributes = res.json()["data"]["items"][0]["nft_data"][0]["external_data"]["attributes"]

for attr in attributes:
    df = df.append({
                    "image_id": 123, 
                    "trait_type": attr["trait_type"], 
                    "value": attr["value"]},
                    ignore_index=True)

df

image_id  trait_type          value
0      123     Clothes  Black Holes T
1      123         Fur          White
2      123        Eyes          Coins
3      123  Background  New Punk Blue
4      123       Mouth    Bored Cigar
5      123         Hat    Spinner Hat

## Get full dataset

In [71]:
df = pd.DataFrame(columns=["image_id", "trait_type", "value"])

for i in range(10_000):
    try:
        URL = f"https://api.covalenthq.com/v1/1/tokens/{BORED_APES_ADDRESS}/nft_metadata/{i}/?quote-currency=USD&format=JSON&key={KEY}"
        res = requests.get(URL)
        
        image_url = res.json()["data"]["items"][0]["nft_data"][0]["external_data"]["image"]
        img_data = requests.get(image_url).content
        with open(f'images//bored_apes/{i}.jpg', 'wb') as handler:
            handler.write(img_data)
            
        attributes = res.json()["data"]["items"][0]["nft_data"][0]["external_data"]["attributes"]

        for attr in attributes:
            df = df.append({
                        "image_id": i, 
                        "trait_type": attr["trait_type"], 
                        "value": attr["value"]},
                        ignore_index=True)
    except:
        print(f"Broke on image {i}")
        df.to_csv(f'bored_apes_{i}.csv')
df.to_csv(f'bored_apes_{i}.csv')

Broke on image 2936
Broke on image 2978
Broke on image 2999
Broke on image 8325


# CRYPTOPUNKS

## Try an example

In [7]:
KEY_NFTPORT = "f3b6580e-c649-4c07-aa3a-f18f1914b4e5"
CRYPTOPUNKS_ADDRESS = "0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb"
i = 100
URL = f"https://api.nftport.xyz/v0/nfts/{CRYPTOPUNKS_ADDRESS}/{i}?chain=ethereum"
res = requests.get(URL, headers={"Authorization": KEY_NFTPORT})

res.json()

{'response': 'OK',
 'nft': {'chain': 'ethereum',
  'contract_address': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb',
  'token_id': '100',
  'metadata_url': None,
  'metadata': {'name': 'CryptoPunk #100',
   'description': None,
   'image': 'https://www.larvalabs.com/cryptopunks/cryptopunk100.png',
   'opensea_cached_image': 'https://lh3.googleusercontent.com/qteV0m0xYrtt9W7dXUiU9dI3Cpaw6K9NzEU0kirxAJGbvNolYkphznaCXyQrThE4Zlj_AEQNXn2RwFILEN3Af6Q',
   'animation': None,
   'opensea_cached_animation': None,
   'external_url': 'https://www.larvalabs.com/cryptopunks/details/100',
   'attributes': [{'display_type': None,
     'max_value': None,
     'order': None,
     'trait_count': 3840,
     'trait_type': 'type',
     'value': 'Female'},
    {'display_type': None,
     'max_value': None,
     'order': None,
     'trait_count': 178,
     'trait_type': 'accessory',
     'value': 'Tassle Hat'},
    {'display_type': None,
     'max_value': None,
     'order': None,
     'trait_count': 126,
  

In [10]:
image_url = res.json()["nft"]["cached_file_url"]
# image_url
Image(url=image_url)

In [11]:
df = pd.DataFrame(columns=["image_id", "trait_type", "value"])

attributes = res.json()["nft"]["metadata"]["attributes"]

for attr in attributes:
    df = df.append({
                    "image_id": i, 
                    "trait_type": attr["trait_type"], 
                    "value": attr["value"]},
                    ignore_index=True)

df

image_id trait_type         value
0      100       type        Female
1      100  accessory    Tassle Hat
2      100  accessory  1 attributes

In [12]:
i = 1
URL = f"https://api.nftport.xyz/v0/nfts/{CRYPTOPUNKS_ADDRESS}?chain=ethereum&page_number={i}&include=metadata"
res = requests.get(URL, headers={"Authorization": KEY_NFTPORT})

In [13]:
res.json()

{'response': 'OK',
 'nfts': [{'chain': 'ethereum',
   'contract_address': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb',
   'token_id': '0',
   'metadata': {'name': 'CryptoPunk #0',
    'description': None,
    'image': 'https://www.larvalabs.com/cryptopunks/cryptopunk0.png',
    'opensea_cached_image': 'https://lh3.googleusercontent.com/loHrK-246FlyQWzT8OONN1tteD2w31sPa3LsUjG_E9Ek2kaSAbodd2plbJKRB5LbwbOtKUzcAcEJEeePCS1lbFB82Q',
    'animation': None,
    'opensea_cached_animation': None,
    'external_url': 'https://www.larvalabs.com/cryptopunks/details/0',
    'attributes': [{'display_type': None,
      'max_value': None,
      'order': None,
      'trait_count': 3840,
      'trait_type': 'type',
      'value': 'Female'},
     {'display_type': None,
      'max_value': None,
      'order': None,
      'trait_count': 271,
      'trait_type': 'accessory',
      'value': 'Green Eye Shadow'},
     {'display_type': None,
      'max_value': None,
      'order': None,
      'trait_count': 245

In [14]:
for nft in res.json()["nfts"]:
    print(nft["token_id"])
    print(nft["cached_file_url"])
    try:
        for attr in nft["metadata"]["attributes"]:
            print(attr["trait_type"], attr["value"])        
    except:
        continue

0
https://storage.googleapis.com/sentinel-nft/raw-assets/12972af803f48315dafa0f8478580a183e55a6c5c4dc2b7aa59e13fd21f34c5b.png
type Female
accessory Green Eye Shadow
accessory Earring
accessory Blonde Bob
accessory 3 attributes
1
https://storage.googleapis.com/sentinel-nft/raw-assets/a200bf39b8c8ec73d1f42aaa579c688a37837090e1eed7cfe461be204448d680.png
10
https://storage.googleapis.com/sentinel-nft/raw-assets/ce23f8010e174e35a4a446156f567267d931af4b43bcf424ea0d358089f52ed7.png
type Female
accessory Mohawk
accessory Blue Eye Shadow
accessory 2 attributes
100
https://storage.googleapis.com/sentinel-nft/raw-assets/45a8440f21b663397a6df365e000b4cc1877dcee1692dae3751e246eb9a0b86b.png
type Female
accessory Tassle Hat
accessory 1 attributes
1000
https://storage.googleapis.com/sentinel-nft/raw-assets/c5b74ad9d8359becc57e9b64624e53972d2111e1a9a360e5bf75b978a5710bba.png
10000
None
1001
https://storage.googleapis.com/sentinel-nft/raw-assets/093002aeecfec29f283561d61491910f9a8e0bfd10ee9f65e64ce3f198

## Full Dataset

In [15]:
df = pd.DataFrame(columns=["image_id", "trait_type", "value"])

for i in range(1, 202):
    URL = f"https://api.nftport.xyz/v0/nfts/{CRYPTOPUNKS_ADDRESS}?chain=ethereum&page_number={i}&include=metadata"
    res = requests.get(URL, headers={"Authorization": KEY_NFTPORT})

    for nft in res.json()["nfts"]:
        nft_id = nft["token_id"]
        try:
            image_url = nft["cached_file_url"]
            img_data = requests.get(image_url).content
            with open(f'images/cryptopunks/{nft_id}.jpg', 'wb') as handler:
                handler.write(img_data)

            for attr in nft["metadata"]["attributes"]:
                df = df.append({
                    "image_id": nft_id, 
                    "trait_type": attr["trait_type"], 
                    "value": attr["value"]},
                    ignore_index=True)
        except:
            continue
    
df.to_csv('cryptopunks.csv')